In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fall-ml2-mipt-2021/sample.csv
/kaggle/input/fall-ml2-mipt-2021/train.csv
/kaggle/input/fall-ml2-mipt-2021/test.csv


In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.svm import SVC, SVR

import xgboost as xgb
from catboost import CatBoostClassifier
import lightgbm as lgb

# Читаем данные

In [5]:
train_data = pd.read_csv('/kaggle/input/fall-ml2-mipt-2021/train.csv', index_col=0)

In [6]:
train_data.head()

,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,...,x_292,x_293,x_294,x_295,x_296,x_297,x_298,x_299,x_300,Category
Id,,,,,,,,,,,,,,,,,,,,,
0,-0.03,-1.00,-0.02,-0.80,-1.37,0.62,-0.09,-0.87,-0.02,0.63,...,-0.16,-0.02,-0.15,-0.00,1.07,-0.06,-0.80,0.39,-0.06,-1.0
1,-0.18,1.71,0.00,-1.09,-1.13,-0.08,2.20,-0.32,0.00,0.81,...,-0.63,0.08,-1.83,0.04,1.49,0.10,0.78,-1.56,0.08,1.0
2,-1.41,0.01,0.00,-0.48,0.31,1.02,0.90,0.41,0.04,-0.00,...,-1.25,0.06,-0.20,0.04,-0.77,0.04,-1.39,0.87,0.02,1.0
3,-0.41,-0.54,-0.02,-0.53,0.52,-1.23,0.62,0.02,-0.03,0.06,...,0.49,-0.00,-0.40,-0.02,-1.02,-0.00,-0.16,0.41,-0.07,-1.0
4,-1.58,0.25,0.02,1.15,-1.95,-0.43,0.40,-0.33,0.02,0.72,...,-0.14,0.04,-0.85,0.00,0.02,0.14,0.68,-1.42,0.02,-1.0


# Преобразование данных

In [7]:
X = train_data.iloc[:, train_data.columns != 'Category'].values
y = train_data.iloc[:, train_data.columns == 'Category'].values.reshape(-1)

X.shape, y.shape

((90000, 300), (90000,))

# Выбор метода

In [ ]:
model = LogisticRegression(solver='liblinear', max_iter=10)

cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=1, random_state=0)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, verbose=1, n_jobs=-1)
print(f'SCORE: {np.mean(n_scores)} +- {np.std(n_scores)}')

In [ ]:
model = AdaBoostClassifier()

cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=1, random_state=0)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, verbose=1, n_jobs=-1)
print(f'SCORE: {np.mean(n_scores)} +- {np.std(n_scores)}')

In [ ]:
model = xgb.XGBClassifier(reg_alpha=20)

cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=1, random_state=0)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, verbose=1, n_jobs=-1)
print(f'SCORE: {np.mean(n_scores)} +- {np.std(n_scores)}')

In [32]:
model = CatBoostClassifier(logging_level='Silent')

cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=1, random_state=0)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, verbose=1, n_jobs=-1)
print(f'SCORE: {np.mean(n_scores)} +- {np.std(n_scores)}')

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.239797 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 55122
[LightGBM] [Info] Number of data points in the train set: 90000, number of used features: 300
[LightGBM] [Info] Start training from score -0.687497
[LightGBM] [Info] Start training from score -0.698830


TypeError: estimator should be an estimator implementing 'fit' method, <lightgbm.basic.Booster object at 0x7fada1f89e10> was passed

In [ ]:
y01 = y
y01[yy01==-1]=0
train_data = lgb.Dataset(X, label=yy01)
param = {'num_leaves': 31, 
         'objective': 'multiclass', 
         'num_class': 2, 
         'metric': ['multi_logloss']}
model = lgb.train(param, train_data, num_boost_round=10)

(model.predict(X).argmax(axis=-1) == y01).mean()

# Подбор гиперапараметров

Лучшую точность показал метод CatBoost. Подберем для нее гиперпараметры.

In [8]:
model = CatBoostClassifier(logging_level='Silent')

In [9]:
param_grid = {'learning_rate': [0.1, 0.3, 0.4], #0.1 was better than 1
                'depth': [8, 10], #8 was better than 6
                'l2_leaf_reg': [0.1, 1, 3]} # 1 was better than 10

grid_search_result = model.grid_search(param_grid, X, y=y, cv=3, partition_random_seed=0,
                                        calc_cv_statistics=True, search_by_train_test_split=True,
                                        refit=True, shuffle=True, stratified=None,train_size=0.8, verbose=True, plot=False)

0:	loss: 0.1438214	best: 0.1438214 (0)	total: 2m 41s	remaining: 45m 48s
1:	loss: 0.1417655	best: 0.1417655 (1)	total: 5m 16s	remaining: 42m 12s
2:	loss: 0.1476600	best: 0.1417655 (1)	total: 7m 48s	remaining: 39m 1s
3:	loss: 0.1406951	best: 0.1406951 (3)	total: 10m 30s	remaining: 36m 47s
4:	loss: 0.1274036	best: 0.1274036 (4)	total: 13m 6s	remaining: 34m 5s
5:	loss: 0.1379908	best: 0.1274036 (4)	total: 15m 37s	remaining: 31m 15s
6:	loss: 0.1412168	best: 0.1274036 (4)	total: 18m 19s	remaining: 28m 48s
7:	loss: 0.1283837	best: 0.1274036 (4)	total: 20m 55s	remaining: 26m 9s
8:	loss: 0.1400087	best: 0.1274036 (4)	total: 23m 29s	remaining: 23m 29s
9:	loss: 0.1522307	best: 0.1274036 (4)	total: 32m 33s	remaining: 26m 2s
10:	loss: 0.1733211	best: 0.1274036 (4)	total: 41m 13s	remaining: 26m 14s
11:	loss: 0.1952503	best: 0.1274036 (4)	total: 49m 45s	remaining: 24m 52s
12:	loss: 0.1442660	best: 0.1274036 (4)	total: 58m 41s	remaining: 22m 34s
13:	loss: 0.1542525	best: 0.1274036 (4)	total: 1h 7m 26s

In [29]:
grid_search_result['params']

{'depth': 8, 'l2_leaf_reg': 1, 'learning_rate': 0.3}

In [15]:
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=1, random_state=0)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, verbose=1, n_jobs=-1)
print(f'SCORE: {np.mean(n_scores)} +- {np.std(n_scores)}')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


SCORE: 0.9381444444444443 +- 0.0018543908520316814


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  7.2min finished


# Обучаем финальную модель

In [23]:
model = CatBoostClassifier(logging_level='Silent', depth=8, l2_leaf_reg=1, learning_rate=0.3, iterations=1500)
model.fit(X, y, logging_level='Silent')

In [24]:
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=1, random_state=0)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, verbose=1, n_jobs=-1)
print(f'SCORE: {np.mean(n_scores)} +- {np.std(n_scores)}')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


SCORE: 0.9383777777777778 +- 0.0014069492431639146


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed: 10.4min finished


In [ ]:
model = CatBoostClassifier(logging_level='Silent', depth=8, l2_leaf_reg=1, learning_rate=0.3, iterations=10000)
param_grid = {} # 1 was better than 10

grid_search_result = model.grid_search(param_grid, X, y=y, cv=3, partition_random_seed=0,
                                        calc_cv_statistics=True, search_by_train_test_split=True,
                                        refit=True, shuffle=True, stratified=None,train_size=0.8, verbose=True, plot=False)

0:	loss: 0.1236998	best: 0.1236998 (0)	total: 24m 58s	remaining: 0us
Estimating final quality...


In [ ]:
grid_search_result

In [ ]:
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=1, random_state=0)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, verbose=1, n_jobs=-1)
print(f'SCORE: {np.mean(n_scores)} +- {np.std(n_scores)}')

# Тестируем финальную модель

In [ ]:
test_data = pd.read_csv('/kaggle/input/fall-ml2-mipt-2021/test.csv', index_col=0)

X_test = test_data.iloc[:, test_data.columns != 'Category'].values

X_test.shape

In [ ]:
predict = model.predict(X_test)

test_data_answer = pd.DataFrame(predict, columns=['Category']).astype(int)

In [ ]:
test_data_answer.head()

# Отправляем модель в лидерборд

In [ ]:
test_data_answer.to_csv('/kaggle/working/predict.csv', index=True, index_label='Id')